# Benchmark 2
---

In [1]:
import matplotlib.pyplot as plt
import numpy             as np
import torch.nn          as nn
import torch

from torch.optim          import Adam
from tqdm                 import tqdm
from astropy              import units, constants
from ipywidgets           import interact

from p3droslo.utils       import convert_angular_to_spatial
from p3droslo.plot        import plot_cube_2D, plot_spectrum
from p3droslo.object      import AstroObject
from p3droslo.observation import DataCube

from p3droslo.model       import TensorModel, SphericallySymmetric
from p3droslo.utils       import print_var, interpolate, convert_angular_to_spatial
from p3droslo.lines       import Line
from p3droslo.loss        import Loss, fourier_loss_1D, fourier_loss_1D

In [2]:
line = Line('CO', 1)

You have selected line:
    CO(J=2-1)
Please check the properties that were inferred:
    Frequency         2.305380000e+11  Hz
    Einstein A coeff  6.910000000e-07  1/s
    Molar mass        28.0101          g/mol


/home/frederikd/.local/lib/python3.9/site-packages/astroquery/lamda/core.py:145: UserWarning: The first time a LAMDA function is called, it must assemble a list of valid molecules and URLs.  This list will be cached so future operations will be faster.
  warnings.warn("The first time a LAMDA function is called, it must "


In [3]:
v_max = 65377.84099979772

In [16]:
model = TensorModel(
    sizes = 559518906748033.94,
    shape = 100,
)

origin_ind    = np.array(model.shape)/2.0 - 0.5
r_z           = model.get_radial_direction(origin=origin_ind)
r             = model.get_radius          (origin=origin_ind)
inverse_r     = r.min() / r

model['log_M'           ] = np.array([np.log(constants.M_sun.si.value)])
# model['log_CO'          ] = np.log(1.0e+19 * inverse_r**2)
# model['log_temperature' ] = np.log(1.0e+3  * inverse_r)
model['log_CO'          ] = np.log(1.0e+10) * np.ones(model.shape)
model['log_temperature' ] = np.log(5.0e+2 ) * np.ones(model.shape)
model['log_v_turbulence'] = np.log(1.5e+2 ) * np.ones(model.shape)
model['velocity_z'      ] = 0.0 * r_z / v_max

model.info()

Variable key:              Free/Fixed:   Field:    Min:           Mean:          Max:
  log_M                      Fixed         False    +6.976e+01     +6.976e+01     +6.976e+01
  log_CO                     Fixed         True     +2.303e+01     +2.303e+01     +2.303e+01
  log_temperature            Fixed         True     +6.215e+00     +6.215e+00     +6.215e+00
  log_v_turbulence           Fixed         True     +5.011e+00     +5.011e+00     +5.011e+00
  velocity_z                 Fixed         True     +0.000e+00     +0.000e+00     +0.000e+00
sizes: (559518906748033.94,)
shape: (100,)


In [17]:
model.dx(0)

5595189067480.34

In [18]:
def get_doppler_shifted_frequencies(velocity_los, frequencies):
    """
    Doppler shifts frequencies given the velocity along the line of sight.
    """
    # Compute the Doppler shift for each cell
    shift = 1.0 + velocity_los * (1.0 / constants.c.si.value)

    # Create freqency tensor for each cell
    freqs = torch.einsum("..., f -> ...f", shift, frequencies)
    
    return freqs


def forward(model, freqs):
    """
    Forward model: image the given TensorModel at the given frequencies.
    """
    nCO    = torch.exp(model['log_CO'])         
    tmp    = torch.exp(model['log_temperature'])
    vel    =   v_max * model['velocity_z']
    v_turb = torch.exp(model['log_v_turbulence'])

    frequencies = get_doppler_shifted_frequencies(vel, freqs)
    # frequencies = get_doppler_shifted_frequencies(vel, freqs)
    
    # Compute the LTE line emissivity and opacity, and the line profile
    eta, chi = line.LTE_emissivity_and_opacity(
        density     = nCO,
        temperature  = tmp,
        v_turbulence = v_turb,
        frequencies  = frequencies
    )
    

    # chi_mod = torch.empty_like(chi)
    # chi_mod[...,  0 , :] = 0.0
    # chi_mod[..., +1:, :] = 0.5 * (chi[..., +1:, :] + chi[..., :-1, :])

    # Create an image of the line
    tau = model.integrate    (chi,                 axis=0)
    img = model.integrate    (eta*torch.exp(-tau), axis=0)
    
    return img


def forward2(model, freqs):
    """
    Forward model: image the given TensorModel at the given frequencies.
    """
    nCO    = torch.exp(model['log_CO'])         
    tmp    = torch.exp(model['log_temperature'])
    vel    =   v_max * model['velocity_z']
    v_turb = torch.exp(model['log_v_turbulence'])

    frequencies = get_doppler_shifted_frequencies(vel, freqs)
    
    # Compute the LTE line emissivity and opacity, and the line profile
    eta, chi = line.LTE_emissivity_and_opacity(
        density     = nCO,
        temperature  = tmp,
        v_turbulence = v_turb,
        frequencies  = frequencies
    )
        
    chi_mod = torch.empty_like(chi)
    chi_mod[...,  0 , :] = 0.0
    chi_mod[..., +1:, :] = 0.5 * (chi[..., +1:, :] + chi[..., :-1, :])

    # Create an image of the line
    tau = model.integrate(chi_mod, axis=0)

    eta_0 = eta[..., :-1, :]
    eta_1 = eta[..., +1:, :]

    exp_minus_tau = torch.exp(-tau)

    emt_0 = exp_minus_tau[..., :-1, :]
    emt_1 = exp_minus_tau[..., +1:, :]

    dtau = tau[..., +1:, :] - tau[..., :-1, :]
    
    mask_a = (dtau > 1.0e-2)
    mask_b = (dtau < 1.0e-2)

    print_var(' eta',  eta)
    print_var(' chi',  chi)
    print_var(' tau',  tau)
    print_var('dtau', dtau)

    eta_0a = eta_0[mask_a]
    eta_0b = eta_0[mask_b]
    eta_1a = eta_1[mask_a]
    eta_1b = eta_1[mask_b]

    emt_0a = emt_0[mask_a]
    emt_0b = emt_0[mask_b]
    emt_1a = emt_1[mask_a]
    emt_1b = emt_1[mask_b]

    dtau_a = dtau[mask_a]
    dtau_b = dtau[mask_b]
    
    term_0a = eta_0a * (emt_1a - emt_0a * (1.0 - dtau_a))
    term_1a = eta_1a * (emt_0a - emt_1a * (1.0 + dtau_a))

    coeff = 1.0 / torch.arange(2,6)
    
    cc     = coeff[0] * torch.ones_like(dtau_b)
    fac_0  = cc.clone() 
    fac_1  = cc.clone()
    cc    *= coeff[1] * dtau_b 
    fac_0 += cc
    fac_1 -= cc
    cc    *= coeff[2] * dtau_b 
    fac_0 += cc
    fac_1 -= cc
    cc    *= coeff[3] * dtau_b
    fac_0 += cc
    fac_1 -= cc

    term_0b = eta_0b * emt_0b * fac_0
    term_1b = eta_1b * emt_1b * fac_1

    result = torch.empty_like(dtau)
    result[mask_a] = (term_0a + term_1a) / dtau_a**2
    result[mask_b] = (term_0b + term_1b)

    img = torch.cumsum(result, dim=0) * model.dx(0)
    
    return img #

In [19]:
model.dx(0) / 1.0e+9 * model.shape[0]

559518.906748034

In [20]:
freqs = line.frequency * (1.0 + 3.0e+3/constants.c.si.value * torch.linspace(-1, +1, 300))

In [21]:
img  = forward (model, freqs)
img2 = forward2(model, freqs)

def plot_spectrum():
    """
    Plot spectrum at a pixel for this observation.
    """
    # Define a plot function
    def plot(i):
        plt.figure(dpi=150)
        # plt.plot(freqs.numpy(), obs[i,j,:].data)
        plt.plot(img [i,:].data, marker='x')
        plt.plot(img2[i,:].data)
        # plt.ylim((1.0e-30, 1.0e+1))
        # plt.yscale('log')
        # plt.axvline(vel(line.frequency)-v_los.si.value, c='k')
        # plt.axvline(vel(line.frequency), c='k')

    # Return an interactive ipywidget
    return interact(plot, i=(0, img2.shape[0]-1))

plot_spectrum()

 eta +1.46e-39 +4.84e-28 +2.91e-27
 chi +1.81e-25 +6.00e-14 +3.60e-13
 tau +0.00e+00 +1.66e+01 +2.00e+02
dtau +1.01e-12 +3.36e-01 +2.02e+00


interactive(children=(IntSlider(value=49, description='i', max=98), Output()), _dom_classes=('widget-interact'…

<function __main__.plot_spectrum.<locals>.plot(i)>

In [ ]:
img.min()

tensor(8.1715e-28, dtype=torch.float64)

In [ ]:
img2.min()

tensor(8.1715e-28, dtype=torch.float64)

In [ ]:
torch.exp(model['log_temperature']).max()

tensor(500.0000, dtype=torch.float64)

In [12]:
torch.exp(model['log_CO']).max()

tensor(1.0000e+10, dtype=torch.float64)

In [52]:
r

array([-2.79756656e+14, -2.79751061e+14, -2.79745465e+14, ...,
        2.79745465e+14,  2.79751061e+14,  2.79756656e+14])